In [1]:
import numpy as np
import pandas as pd
import sklearn

In [2]:
np.random.seed(10)

In [3]:
X = pd.read_excel("X_v3.xlsx").drop(columns=["Unnamed: 0"])
y = pd.read_excel("y_v3.xlsx").drop(columns=["Unnamed: 0"])

In [4]:
from sklearn.feature_selection import SelectKBest, f_regression
select = SelectKBest(f_regression, k=20)

best_features = select.fit_transform(X, y)
feature_scores = pd.concat([pd.Series(X.columns), pd.Series(select.scores_)], axis=1)
feature_scores.columns = ["features", "score"]
features = list(feature_scores.sort_values(by=["score"], ascending=False).head(65)['features'])

X = X[features]
print(X.columns.values)

['WS' 'points' 'Wins Added' 'made_field_goals' 'MPG' 'win_shares'
 'value_over_replacement_player' 'minutes_played' 'poss'
 'attempted_field_goals' 'war_total' 'offensive_box_plus_minus'
 'offensive_win_shares' 'made_free_throws' 'war_reg_season'
 'attempted_free_throws' 'box_plus_minus' 'predator_offense' 'O-LEBRON'
 'raptor_offense' 'player_efficiency_rating' 'defensive_win_shares'
 'war_playoffs' 'games_started' 'win_shares_per_48_minutes' 'turnovers'
 'ORPM' 'RAPM__Off' 'defensive_rebounds' 'usage_percentage' 'RA_EFG__Off'
 'LA_RAPM__Off' 'steals' 'assists' 'made_three_point_field_goals'
 'attempted_three_point_field_goals' 'free_throw_attempt_rate' 'blocks'
 'assist_percentage' 'pace_impact' 'RA_FTR__Off'
 'true_shooting_percentage' 'personal_fouls' 'RA_FTR__Def' 'LA_RAPM__Def'
 'RAPM__Def' 'D-LEBRON' 'RA_EFG__Def' 'games_played' 'offensive_rebounds'
 'Games' 'defensive_rebound_percentage' 'RA_TOV__Off' 'predator_defense'
 'defensive_box_plus_minus' 'total_rebound_percentage'
 'th

C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.preprocessing import Normalizer

In [6]:
from sklearn.model_selection import cross_val_score
def performance(model):
    p = make_pipeline(Normalizer(), model)
    scores = cross_val_score(p, X, y, cv=5, scoring="explained_variance")
    print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
    return scores

## Ensemble

In [7]:
gbr = params = {'n_estimators': 600,
 'min_samples_split': 8,
 'min_samples_leaf': 9,
 'max_features': 'log2',
 'max_depth': 2,
 'learning_rate': 0.05,
 'alpha': 0.55,
  'subsample': 0.75,
 'loss': 'quantile'}
gbr = GradientBoostingRegressor(**params)

In [8]:
etr = ExtraTreesRegressor(max_depth=None, n_estimators=200, min_samples_split=3, min_samples_leaf=2, random_state=42)

In [9]:
rf = RandomForestRegressor(n_estimators=928, criterion='mse', 
                                  max_depth=92, min_samples_split=2, min_samples_leaf=2, min_weight_fraction_leaf=0.0, 
                                  max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                                  bootstrap=True, oob_score=True, n_jobs=None, random_state=None, verbose=0, warm_start=False, 
                                  ccp_alpha=0.0, max_samples=None)

In [10]:
estimators = [
    ("gbr", gbr),
    ("etr", etr),
    ("rf", rf)
]

In [11]:
from sklearn.ensemble import VotingRegressor, StackingRegressor

In [12]:
voting = VotingRegressor(estimators=estimators)
performance(voting)

C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return 

0.66 accuracy with a standard deviation of 0.06


array([0.7030652 , 0.67021849, 0.74523708, 0.55576979, 0.62872897])

In [13]:
stack = StackingRegressor(estimators=estimators, final_estimator=RandomForestRegressor())
performance(stack)

C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return 

0.57 accuracy with a standard deviation of 0.07


array([0.6132685 , 0.63649818, 0.63885247, 0.45553321, 0.52506838])

In [14]:
stack = StackingRegressor(estimators=estimators, final_estimator=ExtraTreesRegressor())
performance(stack)

C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return 

0.57 accuracy with a standard deviation of 0.04


array([0.6223825 , 0.56008695, 0.58215641, 0.5107953 , 0.55020298])

In [15]:
stack = StackingRegressor(estimators=estimators, final_estimator=GradientBoostingRegressor())
performance(stack)

C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return 

0.59 accuracy with a standard deviation of 0.07


array([0.65195003, 0.63162309, 0.63239501, 0.4730435 , 0.55254162])

In [16]:
stack = StackingRegressor(estimators=estimators, final_estimator=voting)
performance(stack)

C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return 

0.62 accuracy with a standard deviation of 0.10


array([0.72432419, 0.63563556, 0.70060434, 0.45291745, 0.59728051])

In [19]:
stack = StackingRegressor(estimators=estimators, final_estimator=stack)
performance(stack)

C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return 

0.57 accuracy with a standard deviation of 0.06


array([0.56713567, 0.6259441 , 0.63199989, 0.46984662, 0.56212234])

In [20]:
stack = StackingRegressor(estimators=estimators, final_estimator=stack)
performance(stack)

C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Rishab\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return 

0.51 accuracy with a standard deviation of 0.12


array([0.6124694 , 0.49305645, 0.67400245, 0.36714067, 0.40269671])